# Relocation Destination Personalization Machination Recommendation

In [3]:
import pandas as pd

In [4]:
# import your quiz results
quiz = pd.read_excel("../data/external/user_responses/RDPM Questionnaire.xlsx", sheet_name="Upload")

In [5]:
# get county data
github_url = "https://raw.githubusercontent.com/rabader/relocation-personalization/main/data/"

FIPS_d = pd.read_csv(github_url + "processed/FIPS_ref.csv").set_index('FIPS')
rel_d = pd.read_csv(github_url + "processed/Religion_dataset.csv").set_index('FIPS')
sd_d = pd.read_csv(github_url + "interim/School_2018_imputed.csv")
ter_d = pd.read_csv(github_url + "processed/Terrain_dataset.csv").set_index('FIPS')
wth_d = pd.read_csv(github_url + "processed/Weather_etc_State_dataset.csv")
acs_d = pd.read_csv(github_url + "interim/ACS_2020_imputed.csv").set_index('FIPS')
hth_d = pd.read_csv(github_url + "interim/health_imputed.csv").set_index('FIPS')
fbi_d = pd.read_csv(github_url + "interim/fbi_imputed.csv").set_index('FIPS')
pol_d = pd.read_csv(github_url + "processed/Politics.csv").set_index('FIPS')
tax_d = pd.read_csv(github_url + "processed/Taxes.csv")

print("FIPS: " + str(len(FIPS_d)), '\n', "Religion: " + str(len(rel_d)), '\n', "School District: " + str(len(sd_d)),
    '\n', "Terrain: " + str(len(ter_d)), '\n', "Weather: " + str(len(wth_d)), '\n', "Census: " + str(len(acs_d)),
      '\n', "Health: " + str(len(hth_d)), '\n', "Crime: " + str(len(fbi_d)), '\n', "Politics: " + str(len(pol_d)),
     '\n', "Taxes: " + str(len(tax_d)))

FIPS: 3153 
 Religion: 3149 
 School District: 18787 
 Terrain: 9356 
 Weather: 50 
 Census: 3221 
 Health: 25125 
 Crime: 18004 
 Politics: 18078 
 Taxes: 51


In [6]:
# add 'Scores' column with 0 penalty points
dfs = [FIPS_d, rel_d, sd_d, ter_d, wth_d, acs_d, hth_d, fbi_d, pol_d, tax_d]

for df in dfs:
    df['Scores'] = 0 # create empty Scores column for all data

In [7]:
# create empty importance list for all data
dfs_str = ['FIPS_d', 'rel_d', 'sd_d', 'ter_d', 'wth_d', 'acs_d', 'hth_d', 'fbi_d', 'pol_d', 'tax_d']

for df in dfs_str:
    exec(df+'_imp = []')

In [8]:
# create grader function (0 penality if at least/most)
def grader(df, response_row, actual):
    
    if quiz.iloc[response_row,1] == "At least":
        score_series = df[actual].apply(lambda x: max(quiz.iloc[response_row,2] - x, 0)**2 * quiz.iloc[response_row,3])
    elif quiz.iloc[response_row,1] == "At most:":
        score_series = df[actual].apply(lambda x: max(x - quiz.iloc[response_row,2], 0)**2 * quiz.iloc[response_row,3])
    else: 
        score_series = abs(quiz.iloc[response_row,2] - df[actual])**2 * quiz.iloc[response_row,3]
    
    imp_score = quiz.iloc[response_row,3] # importance score of this question
    
    return score_series, imp_score

## Religion

In [9]:
# Rel) blanks in this dataset are equal to zero
rel_d = rel_d.fillna(0)

In [10]:
# Rel-1) I prefer the % of the population adhering to any religion to be:
score, imp_score = grader(rel_d, 47, 'All Religious Adherence Rate D') 
rel_d['Scores'] += score 
rel_d_imp.append(imp_score)

In [11]:
# Rel-2) I prefer there to be a significant presence of this religious group:
if quiz.iloc[48,2] != 0:
    rel2_response = rel_d.loc[:,str(quiz.iloc[48,2])]
    rel_d['Scores'] += (((100 - rel2_response)/25)**2) * quiz.iloc[48,3]
rel_d_imp.append(quiz.iloc[48,3]) # imp_score

In [12]:
# add rel scores to FIPS_d score total
FIPS_d['Scores'] = FIPS_d['Scores'].add(rel_d['Scores'], fill_value=0)

## School District

In [13]:
# SD) first questions
sd_cats = ['SchoolDigger Number of Stars Elementary', 'SchoolDigger Number of Stars Middle', 
          'SchoolDigger Number of Stars High', 'Student/Teacher Ratio D', 'Number All Students D']

start_loc = 35
for sd_cat in sd_cats:
    score, imp_score = grader(sd_d, start_loc + sd_cats.index(sd_cat), sd_cat)
    sd_d['Scores'] += score
    sd_d_imp.append(imp_score)

In [14]:
# SD-6) For racial distribution, I prefer there to be AT LEAST this percentage of each race:
races = ['% Asian students', '% Black students', '% Hawaiian Native/Pacific Islander students', '% Hispanic students',
           '% American Indian/Alaska Native students', '% students with Two or More Races', '% White students']

for race in races:
    sd_d['Scores'] += sd_d[race].apply(lambda x: max(((quiz.iloc[races.index(race)+40,2] - x)/25)**2,0) * quiz.iloc[46,3])
sd_d_imp.append(imp_score)

In [15]:
# SD) get lowest penalty score district for each county, reset series to dataframe, reset index
sd_d_grouped = sd_d.groupby(['County','State'])['Scores'].min().to_frame().reset_index() # get district with lowest penalty

In [16]:
# SD) add grouped SD data to FIPS Scores through join
FIPS_d = FIPS_d.reset_index()
sd_d_scores = sd_d_grouped[['County','State','Scores']].rename({'Scores':'sd_Scores'}, axis='columns')
FIPS_d = pd.merge(FIPS_d, sd_d_scores, left_on=['County','State'], right_on=['County','State'], how='left')
FIPS_d['Scores'] += FIPS_d['sd_Scores']
del FIPS_d['sd_Scores']
FIPS_d = FIPS_d.set_index('FIPS')

## Terrain

In [17]:
# Ter) Look at only the most recent completed year but keep entirity for time series analysis
ter_d_all = ter_d.copy()
ter_d = ter_d_all[ter_d['Year']==2011]

In [18]:
# Ter-1) For terrain, I prefer there to be AT LEAST this percentage of each terrain:
terrains = ['Big city', 'Farmland', 'Forests', 'Houses with lots of land', 'Open fields', 'Open water', 
            'Perennial ice/snow', 'Rock/Sand/Clay', 'Suburbia', 'Wetlands']

for terrain in terrains:
    ter_d['Scores'] += ter_d[terrain].apply(
        lambda x: max(((quiz.iloc[terrains.index(terrain)+14,2] - x)/25)**2,0) * quiz.iloc[terrains.index(terrain)+14,3])
ter_d_imp.append(quiz.iloc[terrains.index(terrain)+14,3])

/var/folders/c4/44txy79x4x95gzrg3pt9mxs00000gn/T/ipykernel_36559/3743511843.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ter_d['Scores'] += ter_d[terrain].apply(


In [19]:
# Ter) add ter scores to FIPS_d score total
FIPS_d['Scores'] = FIPS_d['Scores'].add(ter_d['Scores'], fill_value=0)

## State Level Weather

In [20]:
# Wth) first questions
wth_cats = ['Winter Avg temp (F) D', 'Summer Avg temp (F) D', 'Avg Yearly Rainfall (in) D', 'Avg Yearly Snowfall (in) D',
          'Avg Hours Sunshine Daily D', 'Avg Clear Days D', 'Avg Days with Snow D']

start_loc = 28
for wth_cat in wth_cats:
    score, imp_score = grader(wth_d, start_loc + wth_cats.index(wth_cat), wth_cat)
    wth_d['Scores'] += score
    wth_d_imp.append(imp_score)

In [21]:
# Wth) add state Wth data to FIPS Scores through join
FIPS_d = FIPS_d.reset_index()
wth_d_scores = wth_d[['State','Scores']].rename({'Scores':'wth_Scores'}, axis='columns')
FIPS_d = pd.merge(FIPS_d, wth_d_scores, left_on='State', right_on='State', how='left')
FIPS_d['Scores'] += FIPS_d['wth_Scores']
del FIPS_d['wth_Scores']
FIPS_d = FIPS_d.set_index('FIPS')

## Census

In [22]:
# Acs) first questions
acs_cats = ['% Pop Density D', '% Children Under 10 D', '% Children 10 and Older D', '% Couples that are Same-Sex D',
           '% Population Over 25 with at Least a Bachelor Degree D', 
           '% Civilian Population 18 Years and Over that is a Veteran D',
           '% Foreign Born D']

start_loc = 0
for acs_cat in acs_cats:
    score, imp_score = grader(acs_d, start_loc + acs_cats.index(acs_cat), acs_cat)
    acs_d['Scores'] += score
    acs_d_imp.append(imp_score)

In [23]:
# ACS-8) For racial distribution, I prefer the percentage of each race to be AT LEAST:
races = ['% Asian', '% Black', '% Hawaiian or Pacific Islander', '% Hispanic', '% Native American', 
         '% Two or More Races', '% White', '% Other Race']

for race in races:
    acs_d['Scores'] += acs_d[race].apply(lambda x: max(((quiz.iloc[races.index(race)+7,2] - x)/25)**2,0) * quiz.iloc[14,3])
acs_d_imp.append(quiz.iloc[14,3])

In [24]:
# Acs) last questions
acs_cats = ['Median Household Income D', 'Median Gross Rent D', 'Average Commute to Work D']

start_loc = 15
for acs_cat in acs_cats:
    score, imp_score = grader(acs_d, start_loc + acs_cats.index(acs_cat), acs_cat)
    acs_d['Scores'] += score
    acs_d_imp.append(imp_score)

In [25]:
# ACS) add acs scores to FIPS_d score total
FIPS_d['Scores'] = FIPS_d['Scores'].add(acs_d['Scores'], fill_value=0)

## Health

In [26]:
# Hth) Look at only the most recent completed year but keep entirity for time series analysis
hth_d_all = hth_d.copy()
hth_d = hth_d_all[hth_d['Year']==2020]

In [27]:
# Hth) all questions
hth_cats = ['Primary Care Physicians Per 100,000 Population D',
'Mental Health Providers Per 100,000 Population D',
'Dentists Per 100,000 Population D',
'Percent Adults With Limited Access To Doctor Due To Costs D',
'Percent Persons With Limited Access To Healthy Foods D',
'Percent Physically Inactive Persons D',
'Percent Obese Persons Adults D',
'Percent Adults That Report Fair Or Poor Health D',
'Percent Current Adult Smokers D',
'Percent Drinking Adults D',
'STI Rate Per 100,000 Population D',
'Child Mortality Rate Per 100,000 Population D',
'Teen Births Rate Per 1,000 Population D',
'Infant Mortality Rate Per 1,000 Live Births D',
'Percent Low Birthweight Births (<2.5Kg) D']

start_loc = 49
for hth_cat in hth_cats:
    score, imp_score = grader(hth_d, start_loc + hth_cats.index(hth_cat), hth_cat)
    hth_d['Scores'] += score
    hth_d_imp.append(imp_score)

/var/folders/c4/44txy79x4x95gzrg3pt9mxs00000gn/T/ipykernel_36559/2885598990.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hth_d['Scores'] += score


In [28]:
# Hth) add hth scores to FIPS_d score total
FIPS_d['Scores'] = FIPS_d['Scores'].add(hth_d['Scores'], fill_value=0)

## Crime

In [29]:
# FBI) Look at only the most recent completed year but keep entirity for time series analysis
fbi_d_all = fbi_d.copy()
fbi_d = fbi_d_all[fbi_d['Year']==2017]
fbi_d.index = fbi_d.index.astype(int, copy=False) # change index type to int to match FIPS

In [30]:
# FBI) all questions
fbi_cats = ['Violent Crimes Rate D','Property Crimes Rate D']

start_loc = 64
for fbi_cat in fbi_cats:
    score, imp_score = grader(fbi_d, start_loc + fbi_cats.index(fbi_cat), fbi_cat)
    fbi_d['Scores'] += score
    fbi_d_imp.append(imp_score)

/var/folders/c4/44txy79x4x95gzrg3pt9mxs00000gn/T/ipykernel_36559/696417152.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fbi_d['Scores'] += score


In [31]:
# Fbi) add fbi scores to FIPS_d score total
FIPS_d['Scores'] = FIPS_d['Scores'].add(fbi_d['Scores'], fill_value=0)

## Politics

In [32]:
# Pol) blanks in this dataset are equal to zero
pol_d = pol_d.fillna(0)

In [33]:
# Pol) Look at only the most recent completed year but keep entirity for time series analysis
pol_d_all = pol_d.copy()
pol_d = pol_d_all[pol_d['Year']==2020]
pol_d.index = pol_d.index.astype(int, copy=False) # change index type to int to match FIPS

In [34]:
# Pol) I prefer there to be a significant presence of this political group:
if quiz.iloc[66,2] != 0:
    pol_response = pol_d.loc[:,str(quiz.iloc[66,2])]
    pol_d['Scores'] += (((100 - pol_response)/25)**2) * quiz.iloc[66,3]
pol_d_imp.append(quiz.iloc[66,3]) # imp_score

In [35]:
# Pol) add pol scores to FIPS_d score total
FIPS_d['Scores'] = FIPS_d['Scores'].add(pol_d['Scores'], fill_value=0)

## State Taxes

In [36]:
# Tax) all questions
tax_cats = ['State and Mean Local Sales Tax D','Income Tax (Lowest Bracket) D',
            'Income Tax (Highest Bracket) D', 'Median Property Tax D']

start_loc = 67
for tax_cat in tax_cats:
    score, imp_score = grader(tax_d, start_loc + tax_cats.index(tax_cat), tax_cat)
    tax_d['Scores'] += score
    tax_d_imp.append(imp_score)

In [37]:
# Tax) add state tax data to FIPS Scores through join
tax_d_scores = tax_d[['State','Scores']].rename({'Scores':'tax_Scores'}, axis='columns')

FIPS_d = FIPS_d.reset_index()
FIPS_d = pd.merge(FIPS_d, tax_d_scores, left_on=['State'], right_on=['State'], how='left')
FIPS_d['Scores'] += FIPS_d['tax_Scores']
del FIPS_d['tax_Scores']
FIPS_d = FIPS_d.set_index('FIPS')

## Your Top Ten Counties Results

In [38]:
FIPS_d.sort_values(by='Scores', inplace=True)
FIPS_d = FIPS_d[(FIPS_d['State']!='Alaska') & (FIPS_d['State']!='Puerto Rico')] # remove Alaska and Puerto Rico
FIPS_d = FIPS_d.round({'Scores':2})
FIPS_d.head(10)

,County,State,Scores
FIPS,,,
1001,Autauga County,Alabama,0.0
39091,Logan County,Ohio,0.0
39093,Lorain County,Ohio,0.0
39095,Lucas County,Ohio,0.0
39097,Madison County,Ohio,0.0
39099,Mahoning County,Ohio,0.0
39101,Marion County,Ohio,0.0
39103,Medina County,Ohio,0.0
39105,Meigs County,Ohio,0.0


In [39]:
# radar chart of category importance
import plotly.express as px
import numpy as np

categories = ['Census', 'Terrain', 'Weather', 'School District', 'Religion', 'Health', 'Crime', 'Politics', 'Taxes']
importance_means = [np.mean(acs_d_imp), np.mean(ter_d_imp), np.mean(wth_d_imp), np.mean(sd_d_imp), 
                    np.mean(rel_d_imp), np.mean(hth_d_imp), np.mean(fbi_d_imp), np.mean(pol_d_imp),
                    np.mean(tax_d_imp)]

imp_viz_df = pd.DataFrame(dict(Category = categories, Mean_Importance = importance_means))

imp_viz = px.line_polar(imp_viz_df, r='Mean_Importance', theta='Category', line_close=True,
                       title="Mean Importance by Category")
imp_viz.update_traces(fill='toself')
imp_viz.show()

/opt/miniconda3/envs/siads699/lib/python3.10/site-packages/plotly/express/_core.py:271: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trace_data = trace_data.append(trace_data.iloc[0])
